[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/tei.ipynb)

Tento soubor je součástí sestavy elektronických studijních opor [Příběhy dat: Výpočetní přístupy ke studiu kultury a společnosti](https://github.com/CCS-ZCU/pribehy-dat/tree/master). 

# TEI-XML: Digitální edice


**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

V této kapitole se budeme věnovat práci s digitálními editacemi vytvořenými podle standardu [TEI-XML](https://tei-c.org). TEI-XML je způsob, jak digitalizované literární dílo veřejně prezentovat tak, aby z něj zejména badatelé a studenti z oblasti humanitních věd mohli získávat hodnotné informace například skrz pokročilé prohlédávání. Za tímto účelem jsou texty v těchto edicích ve větší či menší míře buď ručně nebo polautomaticky anotované. Tím se myslí, že jsou v těchto textech označeny specifické výrazy, jako jsou názvy osob či organizací, místní názvy nebo jiné důležité termíny. Stejně tak jsou v záhlaví těchto dokumentů standardizovaným způsobem uvedena nejrůznější metadata, typicky data o autorovi daného textu, jeho vydavateli apod. Projektů, které dnes využívají tento standard, je obrovské množství. Rozsáhlý (avšak zdaleka ne úplný) výčet lze nalézt [zde](https://tei-c.org/Activities/Projects/). 

Důkladné obeznámení se s těmito standardy by vydalo na celý samostatný kurz. Zde však budeme postupovat obdobně jako v jiných kapitolách a pouze si uvnitř prostředí Jupyter notebooku prohlédneme, jak některé takovéto digitalizované edice vypadají a jak s nimi můžeme pracovat.

TEI-XML je spojení dvou akronymů: TEI = Text Encoding Initiative a XML = Extensible Markup Language.
 
**XML** je definováno jako *rozšiřitelný značkovací jazyk* a stejnojmenný formát souborů využívaný zejména pro výměnu dat mezi aplikacemi a pro publikování dokumentů (viz [wikipedia](https://cs.wikipedia.org/wiki/Extensible_Markup_Language). V mnoha ohledech se běžné XML dokumenty podobají kódu HTML stránek (jak uvidíme néže, pro práci s nimi se v rámci Pythonu i využívají stejné nástroje). Na rozdíl od HTML stránek však data naformátovaná za využití XML standardu nejsou určena pro webové prohlížeče.

XML soubor sestává z prvků (elementů), které jsou vyznačeny *tagy*, neboli značkami. Tyto tagy jsou vyznačeny pomocí symbolů `<` a `>`, jsou definovány tvůrcem a tvůrce si může vytvářet zcela vlastní tagy. Vždy zde však musí být právě jeden kořenový prvek, tzv. root. V XML je kořenový prvek jedinečný a nemá svůj vlastní obsah, ale pouze obaluje všechny další prvky v dokumentu. 

Tagy mohou a často obsahují atributy, které definují dodatečné vlastnosti příslušných prvků nebo data provazují s dalšími daty, často pomocí hypertextových odkazů. Na rozdíl od HTML musí být v rámci XML všechny tagy tzv. párové. Tag v podobě `<element1>` prvek otvírá a v podobě `</element>` jej uzavírá. Vše mezi tím je obsahem daného prvku. Prvky se do sebe často vnořují, čímž dokument získává určitou stromovou a navigovatelnou strukturu. 

Nejlépe bude struktura xml souboru vidět na následujícím minimalistickém příkladu:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<root>
    <element1 attribute1="value1">Text for element1</element1>
    <element2 attribute2="value2">Text for element2</element2>
</root>
```
Inciativa **TEI** přichází se standardy pro to, jak tento formát dat použít pro zveřejňování dokumentů, jako jsou literární díla, básně, rozhovory, dopisy, nápisy, papyry a mnoho dalšího (existuje např. i verze pro obrazová díla či hudební notaci). TEI byla ustavena v roce 1987 s cílem "vyvíjet, udžovat a propagovat metody pro kódování humanitních dat v elektronické podobě nezávislé na konkrétním hardweru a softwaru ("to develop, maintain, and promulgate hardware- and software-independent methods for encoding humanities data in electronic form" - viz [zde](https://tei-c.org/about/history/)).
 
Veškerý text daného dokumentu se nachází uvnitř prvku `<body></body>`. Ten se může standardizovaným způsoben dále dělit na kapitoly, verše či řádky, či průpovědi jednotlivých mluvčích, v závislosti na žánru daného dokumentu (všechny tyto možnosti jsou detailně popsány v Guidelines [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/index.html)). Stejně tak jsou jasně nadefinovány způsoby, jak v rámci textu vyznačit výskyty jmenných entit, jako jsou jména osob či místní názvy, nebo například datumy (viz [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/ND.html). Tyto standardy si ukážeme na konkrétníh příkladech níže. Pro detailnější obeznámení pak doporučiji například online kurz [zde](https://teach.dariah.eu/course/view.php?id=32). Praktický úvodní text v češtině je dostupný [zde](https://digital-humanities.phil.muni.cz/clanky/metadata-co-ma-spolecneho-netflix-a-digitalni-humanitni-vedy).

## Cvičení

Jak již bylo uvedeno výše, XML dokumenty mají v mnohém obdobné vlastnosti jako webové stránky ve formátu HTML a k práci s nimi lze použít podobné nástroje. Ač TEI-XML dokumenty nejsou určeny pro webové prohlížeče, lze je často nalézt přímo na webu, např. Nezříka jsou také zpřístupňovány pomocí API. V následujícím civčení proto budeme používat v podstatě stejné nástroje jako v případě kapitol o HTTP a websrapingu a využívání API, zejména knihovny `requests` a `BeautifulSoup`.

In [ ]:
import requests
from bs4 import BeautifulSoup

### Korpus německé poezie

Jako první příklad vybereme báseň z korpusu německé poezie **Deutsches Lyrik Korpus** (DLK), dostupného na platformě GitHub [zde](https://github.com/tnhaider/DLK). Jednotlivé básně naformátované ve formě TEI-XML souborů se nachází v této podsložce čítající více než 60,000 položek: https://github.com/tnhaider/DLK/tree/master/DLK/tei/tei_plain.

Vybereme náhodný soubor, otevřeme jej, a zobrazíme jej v "raw" podobě (v oknu souboru klikneme v pravém horním rohu). URL adresu souboru v syrové podobě si zkopírujeme a vložíme jako hodnotu proměnné `url` v buňce
 níže. (Tato url adresa se skládá z informace u umístění souboru v rámci Githubu, která je pro všechny soubory shodná a samotného názvu souboru.)




In [ ]:
url = "https://raw.githubusercontent.com/tnhaider/DLK/master/DLK/tei/tei_plain/dta.poem.10006-Zachariae%2C_Justus_Friedrich_Wilhelm-Die_Vergnügungen_der_Melancholey..tei.xml"
resp = requests.get(url)
resp

Načtený soubor (v proměnné `resp`) následně parsujeme pomocí knihovny `BeuatifulSoup` do proměnné `soup`, pomocí čehož získáme přístup k stromové struktuře daných dat. Obsah si vypíšeme.

In [ ]:
soup = BeautifulSoup(resp.text)
soup

Vidíme kód s jednotlivými prvky. V samotném úvodu vidíme prvky definující metadata o dané básni: jméno autora uvnitř prvku `<persname>`, do sebe vnořuje ještě dva podprvky `<surname>` a `<lastname>`. Všechna tato metadata se nacházejí uvnitř prvku `<teiheader>`. Po jeho uzavření následuje prvek `<text>`, v níž se nachází text příslušné básně. 

Nejprve z textu vyzískejme základní metadata. Stačí nám pohybovat se po stromové struktuře stejně, jako jsme to činili v kapitole o webscrapingu. Například k názvu literárního díla se dostaneme takto:

In [ ]:
soup.teiheader.title

V tomto případě však vidíme i tagy se všemi jejich atributy. Chceme-li samotný textový obsah, musíme použít `text`. Hodnotu si uložíme do proměnné a vypíšeme si ji.




In [ ]:
title = soup.teiheader.title.text
print(title)

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="tei", arg1=title)

Stejně můžeme postupovat v případě dalších prvků:

In [ ]:
author_surname = soup.teiheader.author.surname.text
author_forename = soup.teiheader.author.forename.text

Vypišme si nyní celé jméno autora a příslušné básně.

In [ ]:
print(author_forename, author_surname, "-", title)

Pozor, že některé tagy se zde mohou objevovat opakovaně. Pomocí metody výše jsme si vypsali pouze první výskyt daného tagu. Chceme-li vypsat všechny výskyty, musíme na objekt `soup` použít metodu `findall()`. Vyzkoušíme s prvkem `surname`.

In [ ]:
soup.find_all("surname")

V případě některých básní budeme mít jmen víc. Je to proto, že je zde ještě příjmení editora dané publikace. Proto jsme výše byli konkrétnější a specifikovali, že se v rámci objektu `soup` pohybujeme v prvku autor.

Nyní se přesuňme k textu samotné básně. Ta je v rámci objektu `soup` dostupná v prvku `text`. V nejsyrovější podobě se k textu básně dostaneme takto.

In [ ]:
soup.find("text").get_text() # zde je název tagu "<text>" poněkud matoucí, neboť BeautifulSoup pracuje se stejnojmenným atributem

V rámci digitální edice je však báseň dostupná po jednotlivých verších. Pro jednotlivé verše se používá tag `<l>`. Vytvořme si uspořádaný seznam všech veršů:

In [ ]:
verses = [l.text for l in soup.find("text").find_all("l")]
verses

### Dokumenty EHRI

Jako další příklad použijeme digitální edice vydané projektem [EHRI](https://www.ehri-project.eu) (=The European Holocaust Research Infrastructure). EHRI sestává z mnoha podprojektů. Jedním z nich jsou přepisy raných svědectví přeživších holocaustu ([Early Holocaust Testimony](https://early-testimony.ehri-project.eu)). Tyto edice jsou velice důkladně označkovány ohledně vystujících se jmen osob, místních názvů i organizací, jimž jsou přiřazeny jednoznačné identifikátory, které jsou spravovány v samostatných databázích (vocabularies). Jedná se o vzorový příklad tzv. [Linked Open Data](https://www.ontotext.com/knowledgehub/fundamentals/linked-data-linked-open-data/).    

Vyjdeme  z rozcestníku jednotlivých svědectví dostupného [zde](https://early-testimony.ehri-project.eu/search). Digitální edice ve formátu XML je u každého svědectví dostupná ke stažení přes tlačítko v pravém horním rohu. My však nemusíme soubor fyzicky stahovat, pouze si zkopírujeme URL adresu, na kterou tlačítko pro stažení XML směřuje. 

In [ ]:
url = "https://ehri-editions-et.s3.eu-central-1.amazonaws.com/original/34df1a28893a51d05d5909fb3aa99725.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

Jedná se již o delší dokument. Pro představu o jeho podobě si vypíšeme jeho hlavní části samostatně. Nejprve metadata:

In [ ]:
print(soup.teiHeader.fileDesc)

Vidíme tagy s obdobnými názvy jako u dokumentu výše. Popis je však detailnější. Vidíme zde např. i seznamy osob či místních názvů, které se v dokumentu objevují. Nyní se podívejme na samotný textový obsah. Uložme si tuto část stromu do proměnné `textbody`.

In [ ]:
textbody = soup.find("text")
textbody

V textu vidíme vyznačena jména řady osob.

In [ ]:
persnames = textbody.find_all("persName")
persnames

U některých osob vidíme pod atributem "ref" buď URL adresy do dalších autoritativních databází nebo hashtagem vyznačené identifikátory.

In [ ]:
[pers.get("ref") for pers in persnames]

Obdobně získáme seznam místních názvů:

In [ ]:
placenames = textbody.find_all("placeName")
placenames

Místa jsou zpravidla klasifikována atributem "typ" na "camp", "ghetto" či např. "subjekt". V případě "camp" (koncentrační tábor) zde nalezneme URL odkaz do databáze koncentračních táborů dostupné [zde](https://portal.ehri-project.eu/vocabularies/ehri_camps). V případě jiných místních jmen je zde odkaz na platformu [GeoNames](https://www.geonames.org), se kterou jsme již pracovali v kapitole o websrapingu.    

V některých dokumentech se objevují např. mnohé datumy. V syrovém textu jsou naformátovaná nejrůznějším způsobem, ale v rámci atributu "when" uvnitř tagu jsou dostupná ve strojově čitelné podobě.

In [ ]:
dates = soup.find("text").find_all("date")
dates

In [ ]:
# datumy ve strojově čitelné podobě:
[date.get("when") for date in dates]

Všechny tyto anotace mohou být použity k rozkrývání vztahů mezi dokumenty a nejrůznějším vizualizacím.

### Antická literatura: anglické překlady řeckých textů a řecké originály

Studium antiky obecně, a klasické řecké a latinské literatury zvláště, je dlouhodobě doménou, kde se digitální edice ve formě TEI-XML těší velké oblibě. V současnosti máme takto zpracovanou a volně dostupnou v zásadě veškerou antickou řeckou literaturu, a to díky projektům [Perseus Digital Library](https://www.perseus.tufts.edu/hopper/) a [First1KGreek Project](https://www.perseus.tufts.edu/hopper/), z nichž druhý je pokračováním prvého. Oba projekty průběžně udržují TEI-XML edice jimi zveřejňovaných textů na githubu [zde](https://github.com/PerseusDL/canonical-greekLit) a [zde](https://github.com/OpenGreekAndLatin/First1KGreek). Spolu s řeckými originály jsou v TEI-XML podobě zveřejňovány i anglické překlady.

Podívejme se nyní řecký text Platónova dialogu *Eutyfrón*, patrně jednoho z nejstarších Platónových dialogů, která má relativně přímočarou strukturu: celý text dialogu je tvořen konverzací mezi Sókratem a Eutyfrónem, kteří spolu vedou debatu ohledně podstaty zbožnosti. Text dialogu načteme přímo z Githubu:

In [ ]:
url = "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0059/tlg001/tlg0059.tlg001.perseus-eng2.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

Přímá řeč je zachycena pomocí prvku `<said>`, jehož atribut "who" definuje řečníka. Díky tomu si můžeme celý dialog rozdělit snadno na slova pronesená Sókratem a slova pronesena Eutyfrónem, přičemž můžeme očekávat, že Platónova názorová pozice bude patrně spíše zachycena v Sókratových slovech.

In [ ]:
said_socrates = [said for said in soup.find_all("said") if said.get("who")=="#Socrates"]
len(said_socrates)

In [ ]:
# náhodný výběr Sókratových průpovědí
said_socrates[40:45]

Samotná slova mluvčího se však nacházejí až za tagem `<label>`. Kód tedy ještě upravíme tak, abychom dostali pouze text průpovědi bez tohoto tagu.

In [ ]:
said_socrates = [said.contents[-1] for said in soup.find_all("said") if said.get("who")=="#Socrates"]
said_socrates[40:45]

In [ ]:
said_eutyphro = [said.contents[-1] for said in soup.find_all("said") if said.get("who")=="#Euthyphro"]
len(said_eutyphro)

Může být zajímavé se podívat, jak je v této struktuře zachycen Platónův dialog *Symposion*. Jedná se totiž o dialog nepřímý, vnořený do vcelku komplexního narativního rámce. Dialog je vyprávěný postavou Apollodora v rozhovoru s nejmenovaným společníkem. Hlavní text dialogu je tedy do důsledku vzato přímou řečí, v rámci které se odehrávají další přímé řeči.

In [ ]:
url = "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0059/tlg011/tlg0059.tlg011.perseus-eng2.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

In [ ]:
len(soup.find_all("said"))

In [ ]:
said_apollodorus = [said.contents[-1].text for said in soup.find_all("said") if said.get("who")=="#Apollodorus"]
said_apollodorus[:5]

Uvnitř těchto průpovědí se pak nachází text dialogu samotného, sestávající především z jednotlivých řečí na téma boha Eróta. Takto například získáme Faidrovu oslavnou řeč:

In [ ]:
[said.text for said in soup.find_all("said") if said.get("who")=="#Phaedrus"]

Podíváme-li se na příslušné místo před začátkem této řeči, můžeme zaznamenat, že Faidrova řeč začíná, aniž by byla průvodní Apollodorova řeč ukončena: 
```xml
<p><said who="#Apollodorus" rend="merge">
<milestone ed="P" unit="para"/>First then, as I said, he told me that the speech of Phaedrus began with points of this sort—that Love was a great god, among men and gods a marvel; and this appeared in many ways, but notably in his birth. <milestone unit="section" resp="Stephanus" n="178b"/>
<said who="#Phaedrus" direct="false">Of the most venerable are the honors of this god, and the proof of it is this:  parents of Love there are none, nor are any recorded in either prose or verse...
```

Také si zde povšimněme tagu `<milestone>` a jeho atributů, které vyznačují stránkování v rámci kritické edice, které standardně používáme i při citování z českého překladu.

In [ ]:
kontrola_pruchodu(ntb="tei", arg1=url)

## Rozšiřující analýza: Řecké originály antické literatury

Digitální edice řeckých originálů se v GitHub repozitoři projektu Perseus nachází ve stejných složkách jako výše použité anglické překlady. Tj. v případě dialogu *Eutyfrón: https://github.com/PerseusDL/canonical-greekLit/tree/master/data/tlg0059/tlg001. Všimněme si v této url adrese kódů "tlg0059" a "tlg001". "tlg0059" je standardní identifikátor Platóna napříč různými korpusu, a "tlg001" je taktéž standardní identifikátor dialogu *Eutyfrón* v rámci Platónových děl. Tyto identifikátory proto můžeme použít i jinde. Využijeme zde rozsáhlý automaticky anotovaný korpus řeckých antických textů zvaný GLAUx (k tomu viz [tento](https://aclanthology.org/2021.lchange-1.6/) průvodní text). 

Velká část klasických řeckých textů je nyní dostupná i ve formátu, který jde z jazykového hlediska více do detailu a nabízí texty důkladně morfologicky anotované (tzv. *treebanks* [[wikipedia](https://en.wikipedia.org/wiki/Treebank)]). Tím se v zásadě myslí, že jednotlivá slova jsou zde opatřena svým gramatickým určením a příslušný výraz je zde uveden i v základním tvaru, tak jak jej najdeme ve slovníku (např. veškerá podstatná jména jsou v nominativu singuláru apod.). Nejedná si již v pravém smyslu o digitální edice, neboť zde není věnována pozornost obecným metadatům, která lze snadno dohledat jinde. Ale technologie je v zásadě stejná jako výše.

Podívejme se v této podobě opět na dialog Eutyfrón (povšimněme si číselného kódu v názvu souboru). Omezíme se však pouze na několik prvních vět samotného textu.

In [ ]:
url = "https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/public/xml/0059-001.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

V této podobě budou naši výchozí jednotkou jednotlivé věty. Ty z dokumentu snadno extrahujeme pomocí vyhledání všech výskytů tagu  `<sentences>`. Obratem si vypíšeme 3 jeho první výskyty.

In [ ]:
sents = soup.find_all("sentence")
sents[:3]

Textová data jsou zde zaznamenána tak, že každá věta je zde rozdělena na jednotlivá slova, kterému odpovídá prvek `<word>` v jehož atributech lze nalézt jak slovo v podobě, jak jej najdeme v samotném textu (hodnota atributu "form"), tak ve slovníkové podobě, tzv. "lemma"). Atribut "postag" kóduje tzv. značkování části řeči ("part-of-speech") a atribut "speaker" informaci o mluvčím, tak jak jsme se s ní setkali výše. Všechny tyto atributy jsou velice užitečné a mohou být použity pro nejrůznější jazykové analýzy daných textů, jak z hlediska obsahu, tak stylu či formy. 

Ukážeme si jeden příklad: První písmeno atributu "postag" definuje slovní druh a písmeno "n" značí podstatné jméno ("noun"). Když toto víme, tak pomocí kódu níže z daného textu vypíšeme lemmata všech podstatných jmen, které se v něm vyskytují. 

In [ ]:
nouns = [word["lemma"] for word in soup.find_all("word") if word["postag"][0]=="n"]
nouns[:10]

Slova v této podobě bychom nyní mohli podrobit nejrůznějším kvantitativním analýzám. To si však necháme do samostatné kapitoly, kde si tyto metody ozkoušíme na textových datech v češtině, která však do obdobné podoby budeme zpracovávat jinými způsoby než tak, že bychom je přímo extrahovali z již zveřejněného korpusu.

### Britská poezie 19. století

Pokud nás zajímá poezie, můžeme si samostatně vyzkoušet práci s velice důkladnými digitálními edicemi britské poezie dostupnými [zde](https://www.eighteenthcenturypoetry.org/authors/). Postup bude obdobný jako v předchozích příkladech. 